# 第一章 理解高性能 Python

## 基本的计算机系统
一台计算机的底层组件可被分为三大基本部分:
* 计算单元:计算单元有一个属性告诉我们它每秒能够进行多少次计算
* 存储单元:存储单元有一个属性告诉我们它能保存多少数据,还有一个属性告诉我们能以多快的速度对它进行读写
* 以及两者之间的连接:而连接则有一个属性告诉我们它们能以多快的速度将数据从一个地方移动到另一个地方。



## 通信层

一条总线的主要属性是它的速度:在给定时间内它能传输多少数据。该属性由两个因素决定
* 一次能传输多少数据(总线带宽)和每秒能传输几次(总线频率)。
需要说明的是一次传输的数据总是有序的:一块数据先从内存中读出,然后被移动到另一个地方。
这就是为什么总线的速度可以被拆分为两个因素,因为这两个因素分别独立影响计算的不同方面:
高的总线带宽可以一次性移动所有相关数据,有助于矢量化的代码(或任何顺序读取内存的代码),
而另一方面,低带宽高频率有助于那些经常随机读取内存的代码。
有意思的是,这些属性是由计算机设计者在主板的物理布局上决定的:
* 当芯片之间相距较近时,它们之间的物理链路就较短,就可以允许更高的传输速度。而物理链路的数量则决定了总线的带宽(带宽这个词真的具有物理上的意义!)。

## 理想计算模型和 Python 虚拟机

* 首先是 Python 对象不再是内存中最优化的布局。这是因为Python 是一种垃圾收集语言——内存会被自动分配并在需要时释放。这会导致内存碎片并影响向 CPU 缓存的传输。
* 根源是 Python 的动态类型以及 Python 并不是一门编译性的语言。很多 C 语言开发者已经在多年开发过程中意识到,编译器总是比你聪明。
* 由于 GIL,一次仅有一个核心可以被使用.我们可以使用多进程(multiprocessing 模块)而不是多线程,或者使用 Cython 或外部函数来避免这个问题

# 第二章 通过性能分析找到瓶颈

## 高效地分析性能
* 性能分析的首要目标是对受测系统进行测试来发现哪里太慢(或占用太多 RAM, 或导致太多磁盘 I/O 或网络 I/O)。性能分析一般会导致额外的性能开销

## 计时的简单方法——打印和修饰

In [10]:
%time

from functools import wraps
import time

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        results= fn(*args, **kwargs)
        print("total:",time.time()-t1)
        return results
    return measure_time

@timefn
def my_mul():
    x= [index**3 for index in range(3000000)]
    return x


zz = my_mul()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs
total: 1.0813963413238525


## 用 UNIX 的 time 命令进行简单的计时

* 用 UNIX 的 time 命令进行简单的计时

```
/usr/bin/time -p python your_python.py
```
* 打开--verbose 开关来获得更多输出信息
```
/usr/bin/time --verbose python your_python.py
```

## 使用 cProfile 模块
-s cumulative 开关告诉 cProfile 对每个函数累计花费的时间进行排序,这能让我们看到代码最慢的部分
```
python -m cProfile -s cumulative julia1_nopil.py
```


## 用 runsnakerun 对 cProfile 的输出进行可视化
* runsnake 是一个可视化工具,用于显示 cProfile 创建的统计文件—你只需要看它生成的图像就可以快速意识到哪个函数开销最大

## 用 line_profiler 进行逐行分析
* 输入命令 pip install line_profiler 来安装 line_profiler。用修饰器(@profile)标记选中的函数。用 kernprof.py 脚本运行你的代码,被选函数每一行花费的 CPU 时间以及其他信息就会被记录下来
* line_profiler用来测量 CPU 占用率

## 用 memory_profiler 诊断内存的用量
```
pip install memory_profiler
python -m memory_profiler julia1_memoryprofiler.py
```
* 能够逐行测量内存(RAM)占用率

mprof可视化
```
mprof run memory_profiler_test.py 
mprof plot 
mprof clean
```

## 用 heapy 调查堆上的对象
* 为了使用 heapy,你需要用命令 pip install guppy 安装 guppy 包

## 用 dowser 实时画出变量的实例

## 用 dis 模块检查 CPython 字节码

In [12]:
import dis
dis.dis(my_mul)

  9           0 LOAD_GLOBAL              0 (time)
              2 LOAD_METHOD              0 (time)
              4 CALL_METHOD              0
              6 STORE_FAST               2 (t1)

 10           8 LOAD_DEREF               0 (fn)
             10 LOAD_FAST                0 (args)
             12 LOAD_FAST                1 (kwargs)
             14 CALL_FUNCTION_EX         1
             16 STORE_FAST               3 (results)

 11          18 LOAD_GLOBAL              1 (print)
             20 LOAD_CONST               1 ('total:')
             22 LOAD_GLOBAL              0 (time)
             24 LOAD_METHOD              0 (time)
             26 CALL_METHOD              0
             28 LOAD_FAST                2 (t1)
             30 BINARY_SUBTRACT
             32 CALL_FUNCTION            2
             34 POP_TOP

 12          36 LOAD_FAST                3 (results)
             38 RETURN_VALUE


## 不同的方法不同的复杂度

In [14]:
def fn_expressive(upper = 1000000):
    total = 0
    for n in range(upper):
        total += n
    return total
def fn_terse(upper = 1000000):
    return sum(range(upper))
%timeit fn_expressive()

66.6 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit fn_terse()

21.3 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
import dis
dis.dis(fn_expressive)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (total)

  3           4 SETUP_LOOP              24 (to 30)
              6 LOAD_GLOBAL              0 (range)
              8 LOAD_FAST                0 (upper)
             10 CALL_FUNCTION            1
             12 GET_ITER
        >>   14 FOR_ITER                12 (to 28)
             16 STORE_FAST               2 (n)

  4          18 LOAD_FAST                1 (total)
             20 LOAD_FAST                2 (n)
             22 INPLACE_ADD
             24 STORE_FAST               1 (total)
             26 JUMP_ABSOLUTE           14
        >>   28 POP_BLOCK

  5     >>   30 LOAD_FAST                1 (total)
             32 RETURN_VALUE


In [19]:
dis.dis(fn_terse)

  7           0 LOAD_GLOBAL              0 (sum)
              2 LOAD_GLOBAL              1 (range)
              4 LOAD_FAST                0 (upper)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 RETURN_VALUE


## **在优化期间进行单元测试保持代码的正确性**

# 第三章 列表和元组
* 列表和元组之类的数据结构被称为数组 。
一个数组是数据在某种内在次序下的扁平
列表。这一先验次序十分重要:知道了数据在数组中的确定位置,我们就能以 O(1)
的复杂度得到它!另外,数组可以有多种实现方式。下面是列表和元组的另一个区
别:
* 列表是动态的数组,它们可变且可以重设长度(改变其内部元素的个数)。
* 而元组则是静态的数组,它们不可变,且其内部数据一旦创建便无法改变。
* 元组缓存于 Python 运行时环境,这意味着我们每次使用元组时无须访问内核去分配内存.

## 列表
* Python 2.7 的列表空间分配公式 
```
M = (N >> 3) + (N < 9 ? 3 : 6)
N 是列表的数量
M是列表分配的空间
```

In [2]:
numbers = [5, 8, 1, 3, 2, 6]
numbers.__sizeof__() # 40+ 8*len(numbers)

88

## 元组
* 元组的静态特性的另一个好处体现在一些会在 Python 后台发生的事:资源缓存。Python 是一门垃圾收集语言,这意味着当一个变量不再被使用时,Python 会将该变量使用的内存释放回操作系统,以供其他程序(或变量)使用。然而,对于长度为 1~20 的元组,即使它们不再被使用,它们的空间也不会立刻被还给系统,而是留待未来使用。这意味着当未来需要一个同样大小的新元组时,我们不再需要向操作系统申请一块内存来存放数据,因为我们已经有了预留的内存.
* 元组可以被轻松迅速地创建,因为它们可以避免跟操作系统打交道,而列表很花时间。

In [13]:
t = (1,2,3,4)
print(t.__sizeof__()) #24 +8*len(t)
t=tuple()
print(t.__sizeof__()) #24

56
24


In [14]:
%timeit l = [0,1,2,3,4,5,6,7,8,9]

93.8 ns ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [15]:
%timeit t = (0,1,2,3,4,5,6,7,8,9)

14.6 ns ± 0.869 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


# 第四章 字典和集合
* 字典和集合使用散列表来获得 O(1)的查询和插入。能得到这一效率是因为我们非常聪明地使用散列函数将一个任意的键(如一个字符串或一个对象)转变成了一个列表的索引。散列函数和列表随后可以被用来决定任意数据的位置,而无须搜索。
* Python 对象通常以散列表实现,因为它们已经有内建的__hash__和__cmp__函数。对于数字类型(int 和 float),散列值就是基于它们数字的比特值.

## 字典和命名空间
每当 Python 访问一个变量、函数或模块时,都有一个体系来决定它去哪里查找这些对象。
* 首先,Python 查找 locals()数组,其内保存了所有本地变量的条目。Python 花了很多精力优化本地变量查询的速度,而这也是整条链上唯一一个不需要字典查询的部分。
* 如果它不在本地变量里,那么会搜索 globals()字典。
* 最后,如果对象也不在那里,则搜索__builtin__对象。

注意:
要注意 locals()和 globals()是显式的字典而__builtin__则是模块对象,在搜索__builtin__中的一个属性时,我们其实是在搜索它的 locals()字典(对所有的模块对象和类对象都是如此!)

In [1]:
import math
from math import sin
import dis
def test1(x):
    return math.sin(x)
def test2(x):
    return sin(x)
def test3(x, sin=math.sin):
    return sin(x)

In [5]:
dis.dis(test1)
%timeit test1(123456)

  9           0 LOAD_GLOBAL              0 (math)
              2 LOAD_METHOD              1 (sin)
              4 LOAD_FAST                0 (x)
              6 CALL_METHOD              1
              8 RETURN_VALUE
224 ns ± 5.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
dis.dis(test2)
%timeit test2(123456)

 15           0 LOAD_GLOBAL              0 (sin)
              2 LOAD_FAST                0 (x)
              4 CALL_FUNCTION            1
              6 RETURN_VALUE
181 ns ± 1.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [7]:
dis.dis(test3)
%timeit test3(123456)

 21           0 LOAD_FAST                1 (sin)
              2 LOAD_FAST                0 (x)
              4 CALL_FUNCTION            1
              6 RETURN_VALUE
197 ns ± 5.13 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## 循环内的命名空间查询的降速效果

In [11]:
from math import sin
def tight_loop_slow(iterations):
    """
    >>> %timeit tight_loop_slow(10000000)
    1 loops, best of 3: 2.21 s per loop
    """
    result = 0
    for i in range(iterations):
        # this call to sin requires a global lookup
        result += sin(i)
def tight_loop_fast(iterations):
    """
    >>> %timeit tight_loop_fast(10000000)
    1 loops, best of 3: 2.02 s per loop
    """
    result = 0
    local_sin = sin
    for i in range(iterations):
        # this call to local_sin requires a local lookup
        result += local_sin(i)

In [12]:
%timeit tight_loop_slow(10000000)

1.67 s ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit tight_loop_fast(10000000)

1.49 s ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 第五章 迭代器和生成器

## 生成器的延迟估值
标准库中的 itertools 库

* 它提供了 Python 内建函数 map、 reduce、filter 和 zip 
* 生成器版本在 itertools 中分别是 imap、和 izip、ireduce、ifilter 

In [15]:
from random import normalvariate, random
from itertools import count
def read_data(filename):
    with open(filename) as fd:
        for line in fd:
            data = line.strip().split(',')
            yield map(int, data)
        
def read_fake_data(filename):
    for i in count():
        sigma = random() * 10
        yield (i, normalvariate(0, sigma))
